# FAIRHealth PHT PPDML

In [18]:
import requests
import json
import uuid
import time
from IPython.display import Image
from IPython.core.display import HTML 

import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual, Layout, Button

signalStation = "http://vwdata.personalhealthtrain.net"

def getStations(signalStation):
    response = requests.get(signalStation + "/client")
    return json.loads(response.text)

def defineTrain(image, inputVars, runId):
    return {
        "image": image,
        "inputString": ("%s" % json.dumps(inputVars)),
        "runId": runId
    }

def requestExecution(signalStation, clientId, taskDescription):
    response = requests.post(signalStation + "/client/" + str(clientId) + "/task/add", json=taskDescription)
    taskId = json.loads(response.text)["taskId"]
    return taskId

def getExecutionResult(signalStation, clientId, taskId, wait=True):
    url = signalStation + "/client/" + str(clientId) + "/task/" + str(taskId) + "/result"
    print("Results link:", url)
    jsonResult = json.loads(requests.get(url).text)
     
    while len(jsonResult)==0 & wait:
        time.sleep(2)
        jsonResult = json.loads(requests.get(url).text)
    
    return jsonResult

## Set up for two data stations (two parties)

In [2]:
Image(url= "https://github.com/sunchang0124/PHTDemon/blob/master/Pics/demonPic2.png?raw=true", width=600, height=600)

In [3]:
getStations(signalStation)
# print('You are data party A')

[{'country': 'The Netherlands',
  'email': 'johan.vansoest@maastro.nl',
  'id': 1,
  'institute': 'Maastricht University',
  'ip': '192.168.96.1',
  'last_seen': '2019-05-28 15:08:57.617189',
  'name': 'The Maastricht Study'},
 {'country': 'Netherlands',
  'email': 'johan.vansoest@maastro.nl',
  'id': 2,
  'institute': 'MAASTRO Clinic',
  'ip': '192.168.96.1',
  'last_seen': '2019-05-28 15:08:56.451742',
  'name': 'VWData Station Maastro'}]

## This is Data party A 

In [24]:
def stage_A1(i):
    ### Read data and define your "train" (application) 
    inputA = input_data_A.value.split(';')
    columnsNum = len(inputA)
    item = []
    for i in range(0, columnsNum):
        item.append([int(i) for i in inputA[0].split(',')])

    inputData = {
        "stage": 1,
        "dataA" : item
    }
    print(item)
    myTrain = defineTrain("sophia921025/ppdml_demon", inputData, 1)

    ### Execute the train on client
    clientId = 1
    taskId = requestExecution(signalStation, clientId, myTrain)
    print("Task submitted with ID: %s" % taskId)

    ### Write results for communicating
    results = getExecutionResult(signalStation, clientId, taskId)
    print(results[0]["log"])
    result = json.loads(results[0]["response"])
    print(result.keys())

    return explainPicA()

def explainPicA():
    display(Image(url= "https://github.com/sunchang0124/PHTDemon/blob/master/Pics/demonPic3.png?raw=true", width=600, height=600))
    display(Image(url= "https://github.com/sunchang0124/PHTDemon/blob/master/Pics/demonPic4.png?raw=true", width=600, height=600))
    display(Image(url= "https://github.com/sunchang0124/PHTDemon/blob/master/Pics/demonPic5.png?raw=true", width=600, height=600))
    display(Image(url= "https://github.com/sunchang0124/PHTDemon/blob/master/Pics/demonPic6.png?raw=true", width=600, height=600))


In [38]:
df_A = pd.DataFrame.from_records([[1,2,3],[1,2,4]]).transpose()
df_A.iloc[:,-1]

0    1
1    2
2    4
Name: 1, dtype: int64

In [25]:
#### Demon: Give your data here: ####
style = {'description_width': 'initial'}
uniLayout = Layout(width='50%', height='30px')
input_data_A = widgets.Text(value='1,2,3,4,5;2,3,2,3,2', description='Input data A (comma separate): ', style=style, layout=uniLayout)
# button1 = Button(description="Run", button_style='success', value=None)

interact_manual(stage_A1, i=input_data_A)

interactive(children=(Text(value='1,2,3,4,5;2,3,2,3,2', description='Input data A (comma separate): ', layout=…

<function __main__.stage_A1>

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

## Stage 3 at DMS

In [6]:
def stage_A2(event):
    results = getExecutionResult(signalStation, input_clientID.value, input_taskID.value)
    resultSecond = json.loads(results[0]["response"])

    resultSecond["stage"] = 3
    myThirdTrain = defineTrain("sophia921025/fairhealth_test", resultSecond, 1)

    clientId = 1
    taskId = requestExecution(signalStation, clientId, myThirdTrain)
    print("Task submitted with ID: %s" % taskId)

    resultsThird = getExecutionResult(signalStation, clientId, taskId)
    print(resultsThird[0]["log"])
    resultThird = json.loads(resultsThird[0]["response"])
    print(resultThird.keys())

In [7]:
input_clientID = widgets.BoundedIntText(value=2,max=100,description='Retrieve intermediate result from Client ID:',disabled=False, style=style, layout=uniLayout)
input_taskID = widgets.BoundedIntText(value=150,max=10000,description='Retrieve intermediate result from Task ID:',disabled=False, style=style, layout=uniLayout)
button1 = Button(description="Run", button_style='success', value=None)

button1.on_click(stage_A2)
display(input_clientID, input_taskID, button1)

BoundedIntText(value=2, description='Retrieve intermediate result from Client ID:', layout=Layout(height='30px…

BoundedIntText(value=150, description='Retrieve intermediate result from Task ID:', layout=Layout(height='30px…

Button(button_style='success', description='Run', style=ButtonStyle())

Results link: http://vwdata.personalhealthtrain.net/client/2/task/150/result
Task submitted with ID: 151
Results link: http://vwdata.personalhealthtrain.net/client/1/task/151/result
Stage: 3


=======================INTERNAL FILE LOG===================== 

dict_keys(['randomsSumSet', 'sumNoisesBARand', 'XaTXa'])
